In [ ]:
import math

def entropy(data):
    """
    Calculate the entropy of a dataset.
    """
    num_instances = len(data)  # Get the number of instances in the dataset
    label_counts = {}  # Initialize an empty dictionary to count the occurrences of each class label
    for instance in data:
        label = instance[-1]  # Get the class label of the instance
        if label not in label_counts:
            label_counts[label] = 0  # Initialize the count for this class label if it's not already in the dictionary
        label_counts[label] += 1  # Increment the count for this class label
    
    entropy = 0.0
    for label in label_counts:
        prob = label_counts[label] / num_instances  # Calculate the probability of each class label
        entropy -= prob * math.log2(prob)  # Update the entropy using the formula for entropy
    
    return entropy

def split_data(data, attribute_index):
    """
    Split the dataset based on a given attribute.
    """
    split_data = {}
    for instance in data:
        attribute_value = instance[attribute_index]  # Get the value of the attribute for the current instance
        if attribute_value not in split_data:
            split_data[attribute_value] = []  # Initialize an empty list for this attribute value if it's not already in the dictionary
        split_data[attribute_value].append(instance)  # Add the instance to the list corresponding to its attribute value
    
    return split_data

def information_gain(data, attribute_index):
    """
    Calculate the information gain for a given attribute.
    """
    entropy_before_split = entropy(data)  # Calculate the entropy of the dataset before splitting
    split_data_dict = split_data(data, attribute_index)  # Split the dataset based on the given attribute
    entropy_after_split = 0.0
    
    for attribute_value in split_data_dict:
        subset = split_data_dict[attribute_value]  # Get the subset of data for a particular attribute value
        prob = len(subset) / len(data)  # Calculate the probability of this attribute value
        entropy_after_split += prob * entropy(subset)  # Update the entropy after splitting
    
    information_gain = entropy_before_split - entropy_after_split  # Calculate the information gain
    return information_gain

def best_attribute(data):
    """
    Find the attribute that provides the highest information gain.
    """
    num_attributes = len(data[0]) - 1  # Get the number of attributes (excluding the class label)
    best_gain = -1  # Initialize the best information gain to a negative value
    best_attribute_index = None
    
    for i in range(num_attributes):
        gain = information_gain(data, i)  # Calculate the information gain for each attribute
        if gain > best_gain:
            best_gain = gain  # Update the best information gain if a better one is found
            best_attribute_index = i
    
    if best_attribute_index is not None:
        best_attr = f'Attribute {best_attribute_index + 1}'  # Get the name of the best attribute
        return best_attribute_index, best_attr
    else:
        return None, None

def majority_vote(data):
    """
    Determine the majority class label in a dataset.
    """
    label_counts = {}
    for instance in data:
        label = instance[-1]
        if label not in label_counts:
            label_counts[label] = 0
        label_counts[label] += 1
    
    majority_label = max(label_counts, key=label_counts.get)  # Get the class label with the highest count
    return majority_label

def build_tree(data, attributes):
    """
    Recursively build the decision tree.
    """
    if len(set(instance[-1] for instance in data)) == 1:
        # If all instances have the same class label, return a leaf node
        return data[0][-1]
    
    best_attribute_index, best_attr = best_attribute(data)  # Find the best attribute to split on
    if best_attribute_index is None:
        return majority_vote(data)  # If no best attribute is found, return the majority class label
    
    tree = {best_attr: {}}  # Initialize the tree with the best attribute
    
    split_data_dict = split_data(data, best_attribute_index)  # Split the data based on the best attribute
    for attribute_value in split_data_dict:
        subtree = build_tree(split_data_dict[attribute_value], attributes[:])  # Recursively build subtrees
        tree[best_attr][attribute_value] = subtree
    
    return tree

def train_decision_tree(data):
    """
    Train the decision tree model using the provided dataset.
    """
    attributes = [f'Attribute {i+1}' for i in range(len(data[0]) - 1)]  # Get attribute names
    tree = build_tree(data, attributes)  # Build the decision tree
    return tree

def predict_instance(instance, tree):
    """
    Predict the class label of a single instance using the trained decision tree.
    """
    if isinstance(tree, dict):
        attribute_index = int(list(tree.keys())[0].split()[1]) - 1  # Get the index of the attribute in the instance
        attribute_value = instance[attribute_index]  # Get the value of the attribute in the instance
        subtree = tree[list(tree.keys())[0]][attribute_value]  # Get the subtree corresponding to the attribute value
        return predict_instance(instance, subtree)  # Recursively predict using the subtree
    else:
        return tree  # If a leaf node is reached, return the class label

def predict(instances, tree):
    """
    Predict the class labels of multiple instances using the trained decision tree.
    """
    predictions = []
    for instance in instances:
        prediction = predict_instance(instance, tree)  # Predict the class label for each instance
        predictions.append(prediction)
    return predictions

# Dataset
data = [
    ['Sunny', 'Hot', 'High', 'Weak', 'No'],
    ['Sunny', 'Hot', 'High', 'Strong', 'No'],
    ['Overcast', 'Hot', 'High', 'Weak', 'Yes'],
    ['Rain', 'Mild', 'High', 'Weak', 'Yes'],
    ['Rain', 'Cool', 'Normal', 'Weak', 'Yes'],
    ['Rain', 'Cool', 'Normal', 'Strong', 'No'],
    ['Overcast', 'Cool', 'Normal', 'Strong', 'Yes'],
    ['Sunny', 'Mild', 'High', 'Weak', 'No'],
    ['Sunny', 'Cool', 'Normal', 'Weak', 'Yes'],
    ['Rain', 'Mild', 'Normal', 'Weak', 'Yes'],
    ['Sunny', 'Mild', 'Normal', 'Strong', 'Yes'],
    ['Overcast', 'Mild', 'High', 'Strong', 'Yes'],
    ['Overcast', 'Hot', 'Normal', 'Weak', 'Yes'],
    ['Rain', 'Mild', 'High', 'Strong', 'No']
]

# Train the decision tree
decision_tree = train_decision_tree(data)

# Test the decision tree
test_data = [
    ['Sunny', 'Hot', 'High', 'Weak'],
    ['Overcast', 'Cool', 'Normal', 'Strong'],
    ['Rain', 'Mild', 'High', 'Weak']
]

predictions = predict(test_data, decision_tree)
print("Predictions:", predictions)
